## Airbnb Dataset: 
https://www.kaggle.com/airbnb/seattle/data
### Context
Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Beijing, China.

### Content
The following Airbnb activity is included in this Beijing dataset:

Listings, including full descriptions and average review score
Reviews, including unique id for each reviewer and detailed comments
Calendar, including listing id and the price and availability for that day

### Inspiration
Can you describe the vibe of each Seattle neighborhood using listing descriptions?
What are the busiest times of the year to visit Seattle? By how much do prices spike?
Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle?
For more ideas, visualizations of all Seattle datasets can be found here.

### Acknowledgement
This dataset is part of Airbnb Inside, and the original source can be found here.
(http://insideairbnb.com/get-the-data.html)


## Analysis Procedures (CRISP-DM)


### 1. Business understanding

    - Can you build a recommendation model, by grouping similar listings into one cluster?
    - What are the busiest times of the year to visit Seattle?
    - How the price changes over one year?
    - Can you predict/suggest a price given a listing information? which features impact the price most?
    - Can you predict the review score? What features makes the customers feel more satisfied?
    - * Can you predict the occupancy rate?

    
### 2. Data understanding
    - Data exploration
    
### 3. Data preparation
    - Handling missing values, categorical values, and feature engineering.

### 4. Modelling
    - Use clustering algorithms for grouping/recommendation purpose.
    - Use regression to suggest owner the reseaonable price range.

### 5. Results
    - Answer questions in step 1 'Business understanding' with data visualization.

## 2. Data understanding

In this part, the three datasets in csv format is imported into dataframes using `pandas`, and explored using available pandas functions in below cells. The information obtained about the datasets is summarized as below in this cell.

**Calender dataset** contains the price information for each listing in a calender year. There are 3818 unique `listing_id` in the dataset, and for each `listing_id` there are 365 rows of price corresponding one day between _2016-01-04 and 2017-01-02_ .<br>
The `available` column has two unique values _'t' or 'f'_ meaning _True or False_. When a listing is not available for the day, the columns `price` is _nan_.

**Listing dataset** contains the full description for each listing scraped on _2016-01-04_ , with 3818 rows describing 3818 unique listings. <br>
There are 92 columns/features in this dataset, thus a lot of information that need to be selectively used in later session. <br>
Telling from the column names, the features can be roughly divided into below categories: listing info, host info, location, room/house info, price and booking, reviews, and policies.

**Reviews dataset** contains all the review entries for above mention 3818 listings by _2016-01-03_ since 2009. Each row records the review's info and the detailed text comments for a listing in a certain day, without a numerical score feature. <br>
The positivity of each comments could be predicted through a NLP modelling, but it's not in the objectives of this analysis and the _listing_ dataset already have features of reviews in numerical scores. Thus, the _reviews_ dataset will not be used for furthur analysis.

In [102]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

calender_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/calendar.csv'
listing_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/listings.csv'
reviews_path = '/Users/clairegong/Desktop/UdacityDataScienceNanoDegree/seattle airbnb dataset/reviews.csv'

calender=pd.read_csv(calender_path)
listing=pd.read_csv(listing_path)
reviews=pd.read_csv(reviews_path)


In [103]:
calender.head(5)

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [104]:
calender.listing_id.nunique()

3818

In [105]:
calender.listing_id.value_counts().head()

6752031     365
7404370     365
1259305     365
4672934     365
10310373    365
Name: listing_id, dtype: int64

In [106]:
print('Date range of calender data is between {} and {}.'.format(calender.date.min(),calender.date.max()))

Date range of calender data is between 2016-01-04 and 2017-01-02.


In [107]:
calender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null object
price         934542 non-null object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


In [108]:
# listing.describe()

In [109]:
# listing.info() #3818 entries,92 columns

In [110]:
listing.shape

(3818, 92)

In [111]:
listing.columns # 92 columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [112]:
listing.sample()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
2456,53867,https://www.airbnb.com/rooms/53867,20160104002432,2016-01-04,U District Backyard Cottage & Loft,"Bright, spacious and clean modern backyard cot...","Bright, spacious and clean modern backyard cot...","Bright, spacious and clean modern backyard cot...",none,"Our neighborhood is fun, vibrant and chic, wit...",...,10.0,f,NaN,WASHINGTON,t,moderate,f,f,1,4.52


In [113]:
# listing.iloc[:,:10].sample()

In [114]:
reviews.sample(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
1160,2277640,20737649,2014-10-05,937329,Jennifer,We had a great stay at this spot! Exactly as p...
84251,315120,18667715,2014-08-30,4161253,Jennifer,I absolutely loved staying at Rick's. The spac...
33852,7748866,45597780,2015-09-04,32442148,John,We loved staying here. They were the most frie...
49445,609701,3847593,2013-03-20,511633,Kristina,What beautiful space with all the essentials! ...
12296,5310503,45178275,2015-08-31,41864044,Kelly,"Diana is a friendly, welcoming host. We had a ..."


In [115]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [116]:
print('Date range of review data is between {} and {}.'.format(reviews.date.min(),reviews.date.max()))

Date range of review data is between 2009-06-07 and 2016-01-03.


## 3. Data preparation

The _listing_ and _calender_ datasets will be processed in this section to be ready for comsumption for prediction models. From the information gathered in last part, below processing is in order: data cleaning, feature engineering, missing values imputing, categorical values imputing etc.



**Calender dataset**

In [117]:
#Data cleaning
# remove dollar signs $ and , in the price feature
calender.price = calender.price.replace('[\$,\,]','', regex=True)
calender.price = pd.to_numeric(calender.price)

# modify available feature to boolean values to be meaningful, and convenient for furthur calculation.
calender.available.replace({'t':True, 'f': False}, inplace=True)
calender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null bool
price         934542 non-null float64
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 33.2+ MB


In [118]:
#Feature engineering
# One of our purposes is to see the price trends, considering neighborhoods, so I need to extract the useful 
# neighbourhood features from the listing dataset. After some checking, neighbourhood_group_cleansed feature 
# is the best one, merge this feature to the calender dataset.
neighourhood_group=listing[['id','neighbourhood_group_cleansed']].\
    rename(columns={'id':'listing_id','neighbourhood_group_cleansed':'neighbourhood'})
calender = calender.merge(neighourhood_group, how='left')

calender.sample(5)

,listing_id,date,available,price,neighbourhood
989459,835149,2016-11-08,True,58.0,Capitol Hill
197719,7517215,2016-09-14,True,99.0,Other neighborhoods
239549,413733,2016-04-22,True,150.0,Central Area
1202935,6018416,2016-09-20,True,350.0,Ballard
168582,108468,2016-11-16,True,395.0,Queen Anne


In [119]:
# caculate the yearly occupancy_rate and mean_price for each listing using calender dataset
occupancy_rate=calender.groupby('listing_id').available.mean()
mean_price=calender.groupby('listing_id').price.mean()

### Listing dataset

This dataset has too many(i.e. 92) features, it is known that models trained with too many features are prone to overfit and predict badly on unseen data, thus I need to select those possibly useful in answering our business questions, i.e. predicting prices and scores. Features with descriptive text values is not useful in this context
so I will exclude them all, with the exception for a few with categorical values. Also features that are likely to impact on price and review score but not much, are also discarded.

To select features, the data scientist/analyst need to examine EVERY feature, but a more efficient way I usually do is to examine features by BATCH of 10, or a number of your choice. 

**Examing** includes but is not limited to: checking <br>
- the unique values of categorical values by `nunique` `value_counts`
- the number of missing values by `isnull`
- data distribution of numerical values by `describe`
- comparing features with similar names by `filter`
- and so on


**Below are some features that I discarded as they will provide little information on predicting prices and review scores. Accompanying the feature names are some thoughts written for readers' reference.** <br>
- _experiences_offered_ : 3055/3055 is 'none'
- _host_acceptance_rate_ : 3044/3045 is 100%
- _host_total_listings_count_ : has same info as host_listings_count
- _host_verifications_ : could be useful, by splitting list into elements then into categorical values such as 'has_email', but is not worth the effort in this case.
- _host_has_profile_pic_ : 3809/3817 True
- _square_feet_ : is 97.4% missing using #listing.square_feet.isnull().mean()
- _weekly_price_ : is 47.3% missing, it could be organized to a no_weekly_price feature, but will be discarded in this context.
- _monthly_price_ : is 60.3% missing, reason to discard is as above.
- _calendar_updated_ : values could be unified into number of days ago, but it will be discarded in this context.
- _has_availability_ : 100% True
- _availability_30_ : and it stands for how many days the room is available in the next 30 days. availability_num features follows the same logic. Listing dataset will be used to predict price and score, so these features I think are less significant. To calculate occupancy rate, calender dataset can be used instead.
- _review_scores_xx_ : features I only keep one feature: review_scores_rating as it is representive of overall experience.
- _requires_license_ : 100% False
- _require_guest_profile_picture_ : 3497/3828 False, this could be useful but considered not significant, thus is discarded.
- _require_guest_phone_verification_ : reason to discard is similar to above.
- _host_since_ feature can be used to tell the experience of the host, discarded here as I am not familiar with datetime parsing.

**Codes below are the typical ones I used to examine the feature.**<br>
**Tips added after working a while and coming back: this process is time consume, should use `df.cor` on numerical features for faster elimination.**

In [120]:
listing.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [121]:
# Explore "score" features
scores=listing.filter(like='score')
# I tried to sum the scores in scale of 10, but seems not equals to the 100-scale review_scores_rating.
scores['scores_sum']=listing.filter(like='score').iloc[:,1:].sum(axis=1)
scores[['review_scores_rating','scores_sum']].head(5)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,review_scores_rating,scores_sum
0,95.0,59.0
1,96.0,60.0
2,97.0,60.0
3,NaN,0.0
4,92.0,56.0


In [122]:
listing.reviews_per_month.describe()

count    3191.000000
mean        2.078919
std         1.822348
min         0.020000
25%         0.695000
50%         1.540000
75%         3.000000
max        12.150000
Name: reviews_per_month, dtype: float64

In [123]:
listing.require_guest_profile_picture.value_counts()#.sort_index()

f    3497
t     321
Name: require_guest_profile_picture, dtype: int64

In [124]:
# Codes for examining features by batch of 10 
listing.iloc[:,0:10].sample()
listing.iloc[:,10:20].sample()
listing.iloc[:,20:30].sample()
listing.iloc[:,30:40].sample()
listing.iloc[:,40:50].sample()
listing.iloc[:,50:60].sample()
listing.iloc[:,60:70].sample()
listing.iloc[:,70:80].sample()
listing.iloc[:,80:90].sample()
listing.iloc[:,90:100].sample()

,calculated_host_listings_count,reviews_per_month
2459,3,0.28


### Finally! I have the features I need

In [125]:

features = ['id','host_since','host_response_time','host_response_rate','host_is_superhost','host_listings_count',\
            'host_identity_verified','neighbourhood_group_cleansed','is_location_exact','property_type','room_type',\
           'accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit',\
            'cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews',\
           'review_scores_rating','instant_bookable','cancellation_policy','reviews_per_month']

listing_cleansed=listing[features]
listing_cleansed.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_group_cleansed,is_location_exact,property_type,...,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy,reviews_per_month
0,241032,2011-08-11,within a few hours,96%,f,3.0,t,Queen Anne,t,Apartment,...,NaN,2,$5.00,1,365,207,95.0,f,moderate,4.07
1,953595,2013-02-21,within an hour,98%,t,6.0,t,Queen Anne,t,Apartment,...,$40.00,1,$0.00,2,90,43,96.0,f,strict,1.48
2,3308979,2014-06-12,within a few hours,67%,f,2.0,t,Queen Anne,t,House,...,$300.00,10,$25.00,4,30,20,97.0,f,strict,1.15
3,7421966,2013-11-06,NaN,NaN,f,1.0,t,Queen Anne,t,Apartment,...,NaN,1,$0.00,1,1125,0,NaN,f,flexible,NaN
4,278830,2011-11-29,within an hour,100%,f,2.0,t,Queen Anne,t,House,...,$125.00,6,$15.00,1,1125,38,92.0,f,strict,0.89


### but ... there is much more to be done: manufacture some potentially useful features from available data

In [126]:
# Count of amenties provided is made to a new feature for ease of anaylis.
amenities=listing.amenities.str.split(',')
listing_cleansed['amenities']=amenities.apply(lambda row: len(list(row)))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [127]:
occupancy_rate.shape

(3818,)

In [ ]:
listing_cleansed=listing_cleansed.merge(occupancy_rate,left_index='id', right_index='listing_id')

In [ ]:
# Add the yearly occupancy_rate and mean_price caculated earlier to the listing dataset
occupancy_rate=calender.groupby('listing_id').available.mean()
mean_price=calender.groupby('listing_id').price.mean()

In [27]:
listing_cleansed.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_group_cleansed,is_location_exact,property_type,...,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,instant_bookable,cancellation_policy,reviews_per_month
0,241032,2011-08-11,within a few hours,96%,f,3.0,t,Queen Anne,t,Apartment,...,0,2,$5.00,1,365,207,95.0,f,moderate,4.07
1,953595,2013-02-21,within an hour,98%,t,6.0,t,Queen Anne,t,Apartment,...,$40.00,1,$0.00,2,90,43,96.0,f,strict,1.48
2,3308979,2014-06-12,within a few hours,67%,f,2.0,t,Queen Anne,t,House,...,$300.00,10,$25.00,4,30,20,97.0,f,strict,1.15
3,7421966,2013-11-06,NaN,NaN,f,1.0,t,Queen Anne,t,Apartment,...,0,1,$0.00,1,1125,0,NaN,f,flexible,NaN
4,278830,2011-11-29,within an hour,100%,f,2.0,t,Queen Anne,t,House,...,$125.00,6,$15.00,1,1125,38,92.0,f,strict,0.89


In [28]:
listing_cleansed.shape

(3818, 29)

### Fomating the feature values
includes ...
- Remove the %, $ unit symbols from value features, convert format to numerical types.
- Convert f/t binary features values to binary features using 0/1
- Fill missing values
- Drop duplicates
- Get dummies of categorical values
- Scaling

In [29]:
for feature_with_symbol in ['cleaning_fee','extra_people','host_response_rate','price','security_deposit']:
    no_symbol = listing_cleansed[feature_with_symbol].replace('[\$,\%]','', regex=True)
    listing_cleansed[feature_with_symbol] = pd.to_numeric(no_symbol)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# For features having two categories only, knowing one indicates the other, thus replace them with '1'&'0', 
#no need one-hot encoding.
binary_features=['host_is_superhost','host_identity_verified', 'is_location_exact', 'instant_bookable']
listing_cleansed[binary_features]=listing_cleansed[binary_features].replace({'t':1, 'f':0})

In [ ]:
listing_cleansed[['security_deposit','cleaning_fee']]=\
    listing_cleansed[['security_deposit','cleaning_fee']].fillna(0.0)

In [57]:
#handling missing values
# target features 'price' no missing values, 'review_scores_rating' has 647 missing values, later need to drop 
# rows for review_scores_rating is missing
listing_cleansed.isnull().sum()

# check 0 dulicates
listing_cleansed.duplicated().sum()
listing_cleansed.duplicated(subset='id').sum()

# fill in others columns with missing values by 'mode' for categorical values, and by 'mean' for numerical values
categorical_features=listing_cleansed.select_dtypes(include=['object']).columns
numerical_features=listing_cleansed.select_dtypes(exclude=['object']).columns

from sklearn.impute import SimpleImputer
imp_med=SimpleImputer(strategy='median')
imp_mode=SimpleImputer(strategy='most_frequent')

listing_filled[categorical_features]=pd.DataFrame(imp_mode.fit_transform(listing_cleansed[categorical_features]))
listing_filled[numerical_features]=pd.DataFrame(imp_med.fit_transform(listing_cleansed[numerical_features]))
listing_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 28 columns):
host_response_time              3818 non-null object
neighbourhood_group_cleansed    3818 non-null object
property_type                   3818 non-null object
room_type                       3818 non-null object
bed_type                        3818 non-null object
cancellation_policy             3818 non-null object
id                              3818 non-null float64
host_response_rate              3818 non-null float64
host_is_superhost               3818 non-null float64
host_listings_count             3818 non-null float64
host_identity_verified          3818 non-null float64
is_location_exact               3818 non-null float64
accommodates                    3818 non-null float64
bathrooms                       3818 non-null float64
bedrooms                        3818 non-null float64
beds                            3818 non-null float64
amenities                       381

In [88]:
# now data is processed we can furthur eliminate more  features by checking (numerical) feature correlations 
#with target variables 'price' and 'review_scores_rating' using `corr()` function
price_corr=listing_cleansed.corr()[['price']].apply(lambda x: abs(x)).sort_values(by=['price'],ascending=False)
review_corr

,price
price,1.000000
accommodates,0.652218
bedrooms,0.627720
beds,0.589525
cleaning_fee,0.576423
bathrooms,0.516424
guests_included,0.392875
security_deposit,0.333579
reviews_per_month,0.218588
amenities,0.184176


In [60]:
# one hot encode categorical values
listing_encoded=pd.get_dummies(listing_filled)


In [68]:
listing_encoded.columns.shape

(70,)